In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import glob
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.externals import joblib
from skimage.transform import resize
from tqdm import tqdm_notebook as tqdm
import ipywidgets as ipy

from common_blocks.utils import plot_list, read_images
from common_blocks.metrics import compute_ious, compute_eval_metric

METADATA_FILEPATH = 'YOUR/metadata.csv'
METADATA_FILEPATH = '/mnt/ml-team/minerva/open-solutions/salt/files/metadata.csv'

OUT_OF_FOLD_TRAIN_RESULTS_FILEPATH = 'YOUR/validation_results.pkl'
OUT_OF_FOLD_TRAIN_RESULTS_FILEPATH = '/mnt/ml-team/minerva/open-solutions/salt/files/out_of_fold_predictions/sal_2173_cv_851_lb_848//out_of_fold_train_predictions.pkl'

In [ ]:
def load_img(path):
    img = np.array(Image.open(path))
    return img

def filter_iout(results, iout_range):
    iout_min, iout_max = iout_range
    results_filtered = []
    for tup in results:
        if iout_min<=tup[0]<=iout_max:
            results_filtered.append(tup)
    return results_filtered

def filter_size(results, size_range):
    size_min, size_max = size_range
    results_filtered = []
    for tup in results:
        if size_min<=tup[1]<=size_max:
            results_filtered.append(tup)
    return results_filtered

In [ ]:
metadata = pd.read_csv(METADATA_FILEPATH)

oof_train = joblib.load(OUT_OF_FOLD_TRAIN_RESULTS_FILEPATH)
ids = oof_train['ids']
predictions = oof_train['images']

In [ ]:
THRESHOLD = 0.5

predicted_maps, predicted_masks, masks, images, depths, sizes = [],[],[],[],[],[]
for idx, pred in tqdm(zip(ids, predictions)):
    row = metadata[metadata['id']==idx]
    predicted_map = resize(pred[1,:,:],(101,101),mode='constant')
    predicted_mask = (predicted_map > THRESHOLD).astype(int)
    image = load_img(row.file_path_image.values[0])
    mask = (load_img(row.file_path_mask.values[0]) > 0).astype(int)
    depth = row.z.values[0]
    size = np.sum(mask)
    
    images.append(image)
    masks.append(mask)
    depths.append(depth)
    predicted_maps.append(predicted_map)
    predicted_masks.append(predicted_mask)
    sizes.append(size)

In [ ]:
iouts = [compute_eval_metric(gt, pred) for gt, pred in tqdm(zip(masks, predicted_masks))]
results = list(zip(iouts, sizes, depths, images, predicted_masks, predicted_maps, masks))

# Score by size

In [ ]:
print('IOUT {:.4f}\n'.format(np.mean(list(zip(*results))[0])))
for size_range in [(0,0),(1,300),(300,1000),(1000,3000),(3000,9000), (9000,10201)]:
    results_by_size = filter_size(results, size_range)
    iout = np.mean(list(zip(*results_by_size))[0])
    sample_size = len(results_by_size)
    fraction = len(results_by_size)/len(results)
    print('size {} | IOUT {:.4f} | sample nr {} | fraction {} | max gain {:.4f}'.format(size_range, 
                                                                       iout,
                                                                       sample_size, 
                                                                       fraction,
                                                                       (1.0-iout) * fraction
                                                                       ))

# Predicted mask exploration

In [ ]:
results_filtered = results.copy()
results_filtered = filter_iout(results_filtered, iout_range=(0.0,0.2))
results_filtered = filter_size(results_filtered, size_range=(1,300))

print('sample nr {} fraction {} mean IOUT {}'.format(len(results_filtered), 
                                                     len(results_filtered)/len(results),
                                                     np.mean(list(zip(*results_filtered))[0])))

@ipy.interact(idx = ipy.IntSlider(min=0,max=len(results_filtered)-1,value=0,step=1))
def present(idx=idx):
    iout, s, z, img, pred_mask, pred_map, gt = results_filtered[idx]
    print('IOUT {}, size {}, depth {}'.format(iout, s, z))
    plot_list(images=[img, pred_map],labels=[pred_mask, gt])